In [ ]:
import urllib3
import json
import certifi
import pandas as pd
import numpy as np
import time
import string
import openpyxl
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
subReddit = 'cars'

In [ ]:
#loop due to reddit 100 post limit
for i in range(10):
    #reddit only accepts one request every 2 seconds, adds a delay between each loop
    time.sleep(5)

    #set URL based on loop
    if i == 0:
        requestUrl = 'http://www.reddit.com/r/' + subReddit + '/.json?sort=top&t;=thisweek&limit;=100'
    else:    
        last = rawData[-1]['data']['name']
        requestUrl = 'https://www.reddit.com/r/' + subReddit + '/.json?sort=top&t;=thisweek&limit;=100&after;=%s' % last

    #actual payload get
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    payload = http.request('GET', requestUrl)
    rawDataTemp = json.loads(payload.data.decode('utf-8'))

    #create initial json data or append
    if i == 0:
        rawData = rawDataTemp['data']['children']
    else:    
        rawData += rawDataTemp['data']['children']

In [ ]:
dataNames = [value for value in rawData[0]['data']]

In [ ]:
tmpList = []
for i in range(0, len(rawData)):
    dictionary = {}
    for names in dataNames:
        try:
            dictionary[str(names)] = rawData[i]['data'][str(names)]
        except:
            dictionary[str(names)] = 'None'
    tmpList.append(dictionary)
df=pd.DataFrame(tmpList)

In [ ]:
#prep string to tokenize
df.drop(df.columns.difference(['title']), 1, inplace=True)
df['title'] = df['title'].str.lower()
df['title'] = df['title'].str.replace('[^a-zA-Z]', ' ')
df['title'] = df['title'].str.replace('[^\w\s]+', ' ')

tokens = [word_tokenize(i) for i in df['title']]

In [ ]:
#clean tokenized data
dfTokens = pd.DataFrame(tokens)
words = pd.melt(dfTokens)
words.drop(columns=['variable'], inplace=True)
words['value'] = words['value'].str.strip()
words.dropna(inplace=True)

In [ ]:
# filter out stop words
stopWords = set(stopwords.words('english'))
finalWordList = [w for w in words['value'] if not w in stopWords]

In [ ]:
finalWordList = pd.DataFrame(finalWordList)
finalWordList.to_excel('CleanWordList.xlsx', engine='openpyxl')